In [2]:
from torch import cuda
import os
from training_code import *
from labels_to_ids import task5_labels_to_ids
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import csv
os.environ["CUDA_VISIBLE_DEVICES"]="0"
device = 'cuda' if cuda.is_available() else 'cpu' #save the processing time

In [ ]:
class prediction_Info:
    def __init__(self, model_load_location, dataset_location):
        model = AutoModelForSequenceClassification.from_pretrained(model_load_location)
        model.to(device)
        dev_data = read_task5(dataset_location , split = 'dev')
        tokenizer = AutoTokenizer.from_pretrained(model_load_location)
        labels_to_ids = task5_labels_to_ids
        dev_loader = initialize_data(tokenizer, (256, 32), dev_data, labels_to_ids, shuffle = False)
        labels_dev, predictions_dev, dev_accuracy = testing(model, dev_loader, labels_to_ids, device)

        self.model = model
        self.location = model_load_location
        self.dataset_location = dataset_location
        self.labels = labels_dev
        self.predictions = predictions_dev
    
    def determine_confusion(self):
        true_negative, true_positive, false_negative, false_positive = 0, 0, 0, 0
        for i in range(len(self.labels)):
            label = self.labels[i]
            guess = self.predictions[i]
            if label == '0' and guess == '0':
                true_negative += 1
            elif label == '0' and guess == '1':
                false_positive += 1
            elif label == '1' and guess == '0':
                false_negative += 1
            elif label == '1' and guess == '1':
                true_positive += 1
        
        precision = (true_positive) / (true_positive + false_positive)
        recall = (true_positive) / (true_positive + false_negative)
        F1_score = (2) / (  (1/precision) + (1/recall) )

        self.confusion_matrix = [[true_negative, false_positive], [false_negative, true_positive]]
        print(self.confusion_matrix)
        return precision, recall, F1_score

if __name__ == "__main__":
    """Calculating F1 Score"""
    # for i in range(5):
    #     precision, recall, F1 = prediction_Info(f'../../saved_models/roberta-base/task7/{i}', '../Datasets/TASK7/').determine_confusion()
    #     print(f"model {i}: ", precision, recall, F1)
    
    """Write txt file"""
    model2_preds = prediction_Info(f'../../saved_models/roberta-base/task7/2', '../Datasets/TASK7/').predictions
    filename = '../Datasets/TASK7/dev.csv'
    answer = ["tweet_id\tlabel"]
    with open(filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for i, row in enumerate(csv_reader):
            if i > 0:
                tweet_id = row[0]
                answer.append(f"{tweet_id}"+"\t"+f"{model2_preds[i-1]}")
    print(answer)

    with open('../Datasets/TASK7/answer.txt', 'w') as f:
        for line in answer:
            f.write(line)
            f.write('\n')